In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interactive

import folium
import folium.plugins as plugins
from folium.plugins import HeatMap

# Read in data
df = pd.read_csv("../wa_collisions/data/Collisions.csv")

C:\Users\salik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2683: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.roadcond.value_counts()

Dry               112958
Wet                43634
Unknown            14474
Ice                 1112
Snow/Slush           913
Other                116
Standing Water       109
Sand/Mud/Dirt         71
Oil                   56
Name: roadcond, dtype: int64

In [3]:
# As shown above, rearranging of the weather column may be necessary. For instance merge NULL with unknown, 
# shorten the levels with too many words.

#### Example Code to create an interactice widget

In [7]:
# Define widget
roadcond_selection = widgets.Dropdown(
    options=list(set(df[-pd.isnull(df.roadcond)].roadcond.values)),
    value='Dry',
    description='roadcond:',
    disabled=False
)

# Define function to view or plot data
def view(roadcond=''):
     print(df[df.roadcond == roadcond].shape)

# Define interactivity
interactive(view, roadcond=roadcond_selection)

#### Create an interactive widget to plot heatmap of collisions by weather type

In [22]:
# Define widget
weather_selection = widgets.Dropdown(
    options=list(set(df[-pd.isnull(df.weather)].weather.values)),
    value='Snowing',
    description='Weather:',
    disabled=False
)

columns = ['X', 'Y', 'weather']

# Define function to view or plot data
def map_by_weather(weather=''):
    df_collision = df.loc[df.weather == weather, columns].dropna(axis = 0, how = 'any')
    coordinates = df_collision.loc[:, ['Y', 'X']].values
    NewData = coordinates * np.array([[1, 1]])
    
    data = list()
    data.append(NewData.tolist())
    m = folium.Map(location=[47.706850, -122.333961], 
                   tiles='Stamen Toner', 
                   zoom_start=11)
    HeatMap(data[0]).add_to(m)
    display(m)
    
# Define interactivity
interactive(map_by_weather, weather=weather_selection)

A Jupyter Widget

#### Convert the codes above into one function

#### TO-DO: toggle by neighborhoods

In [ ]:
def visualize_neighborhood(neighborhood_data, mapping_value):
    """
    Visualizes the data provided per neighborhood in abs
    folim map nad returns the map.
    Args:
        neighborhood_data(pandas dataframe): Dataframe containing
            the data value per neighborhood which is to be mapped.
        mapping_value (string): Name of column of neighborhood_data
            to be mapped.
    Returns:
        the map produced
    Raises:
        ValueError: if neighborhood_data doesn't have the column
            'object_id' or mapping_value
    """
    if not 'object_id' in neighborhood_data.columns:
        raise ValueError("Dataframe doesn't have a column object_id")
    if not mapping_value in neighborhood_data.columns:
        raise ValueError("Dataframe doesn't have a column " + mapping_value)

    neigborhood_map = folium.Map(
        location=MAP_LOCATION_START,
        zoom_start=MAP_ZOOM)

    # Finding maximum and minimum values for range
    # Adding a small error to max as folium is not good at comparing floats
    max_value = max(neighborhood_data[mapping_value]) + ERROR
    min_value = min(neighborhood_data[mapping_value])

    neigborhood_map.choropleth(
        geo_data=MAP_JSON,
        data=neighborhood_data,
        columns=['object_id', mapping_value],
        key_on='feature.properties.OBJECTID',
        threshold_scale=list(np.linspace(min_value, max_value, 6)),
        fill_color='YlOrRd',
        highlight=True)
    return neigborhood_map